In [1]:
from wmb import cemba
import pandas as pd
from ALLCools.dmr.rms_test import permute_root_mean_square_test_and_estimate_p, downsample_table
import joblib
import pathlib
from tqdm import tqdm
from ALLCools.mcds import RegionDS
import xarray as xr
from collections import defaultdict
import joblib

In [2]:
celltype_dmr_zarr_path =  '/cemba/wmb/genome/DMR/CEMBA.snmC.AllGroupedDMRs.CellType.mC.zarr/'

In [3]:
mc_ds = xr.open_zarr(celltype_dmr_zarr_path)
_ds = [mc_ds]
dmr_ds = RegionDS(xr.merge(_ds))
dmr_ds.region_dim = 'dmr'

## get all dmrs

In [4]:
all_dmrs = dmr_ds.get_index('dmr')

In [5]:
all_dmr_groups = defaultdict(list)
for dmr in all_dmrs:
    group, length =dmr.split(':')
    all_dmr_groups[group].append(dmr)   
    

In [6]:
use_dmrs = []
for group, dmrs in all_dmr_groups.items():
    dmr_lengths = {}
    for dmr in dmrs:
        length = int(dmr.split(':')[1].split('-')[1])
        dmr_lengths[dmr] = length
    longest_dmr = max(dmr_lengths, key=lambda k: dmr_lengths[k])
    use_dmrs.append(longest_dmr)


In [7]:
assert len(use_dmrs) == len(all_dmr_groups)

In [8]:
len(use_dmrs)

2564356

In [9]:
joblib.dump(use_dmrs, 'use_dmrs_list')

['use_dmrs_list']

In [31]:
chrs_list= []
dmr_start_list= []
dmr_end_list = []

for dmr in use_dmrs:
    group, start_length = dmr.split(':')
    _chr, group_start = group.split('-')
    dist_to_group_start, length = start_length.split('-')
    dmr_start = int(group_start) + int(dist_to_group_start)
    dmr_end = dmr_start + int(length)
    chrs_list.append(_chr)
    dmr_start_list.append(dmr_start)
    dmr_end_list.append(dmr_end)

In [33]:
all_dmr_df = pd.DataFrame({'chrom':chrs_list,'start':dmr_start_list,'end':dmr_end_list})

In [34]:
all_dmr_df.to_csv('CEMBA.mC.grouped-DMR.bed', sep = '\t', header = None, index= None)

In [35]:
! pwd

/home/qzeng_salk_edu/project/cemba/230830_DMR_DAR


In [38]:
all_dmr_df

,chrom,start,end
0,chr1,3003581,3003583
1,chr1,3003639,3003641
2,chr1,3003720,3003899
3,chr1,3005997,3005999
4,chr1,3006781,3006783
...,...,...,...
2564351,chrX,170844078,170844080
2564352,chrX,170844465,170844467
2564353,chrX,170849226,170849228
2564354,chrX,170854561,170854563
